In [1]:
import os
%load_ext autoreload
%autoreload 2

from FlexivPy.joy import XBoxController
joy = XBoxController(0)
joy.getStates()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


Authorization required, but no authorization protocol specified

error: XDG_RUNTIME_DIR is invalid or not set in the environment.


Put your stick at reset and do not touch it while calibrating


{'left_joy': array([ 0., -0.]),
 'right_joy': array([ 0., -0.]),
 'left_trigger': 0.0004999999999999449,
 'right_trigger': 0.0004999999999999449,
 'A': 0,
 'B': 0,
 'X': 0,
 'Y': 0,
 'left_bumper': 0,
 'right_bumper': 0,
 'options_left': 0,
 'options_right': 0,
 'left_joy_btn': 0,
 'right_joy_btn': 0}

Before going through the next cells, start the asynchronous simulator or the real robot bridge by tunning the following in your terminal:

**Asynchronous Simulator:**

```bash
flexivpy_async_sim --mode velocity
```

**Real Robot Client**

```bash
robot_server -cm 3 -g --path /home/FlexivPy/FlexivPy/assets/ -rcf /home/FlexivPy/flexivpy_bridge/config.yaml
```
The `cm` 3 means that the robot is started in joint velocity mode.

In [ ]:
import pinocchio as pin
import numpy as np
from FlexivPy.robot.model.pinocchio import FlexivModel
from FlexivPy.robot.interface import FlexivDDSClient
import time
from FlexivPy.robot.dds.subscriber import get_last_msg
import cv2
from cyclonedds.domain import DomainParticipant
from cyclonedds.topic import Topic
from cyclonedds.sub import Subscriber, DataReader
import time

from cyclonedds.domain import DomainParticipant
from cyclonedds.topic import Topic
from FlexivPy.robot.dds.flexiv_messages import EnvImage
from datetime import datetime
import matplotlib.pyplot as plt


class CameraDDSClient:
    def __init__(self):

        self.domain_participant = DomainParticipant(10)

        self.topic_env_image = Topic(self.domain_participant, "EnvImage", EnvImage)
        self.subscriber_env_image = Subscriber(self.domain_participant)
        self.reader_env_image = DataReader(
            self.subscriber_env_image, self.topic_env_image
        )

        self.warning_no_env_image_dt = 0.2
        self.warning_no_env_image_send = 0.2

        self.max_wait_time_first_msg = 20

        

        # create a smiluation in another process
        self.last_img = None

        tic = time.time()
       
        self.time_last_img = tic

        tic = time.time()
        print("waiting to receive the image ...")
        while True:
            if self.is_ready():
                print("Camera is ready!")
                break
            if time.time() - tic > self.max_wait_time_first_msg:
                raise Exception("Camera is not ready! -- Start the server first!")
            time.sleep(.1)

    def is_ready(self):
        return self.get_env_image() is not None


    def get_env_image(self):
        tic = time.time()
        msg = get_last_msg(self.reader_env_image, EnvImage)
        if msg:
            now = datetime.now()  # TODO: avoid calling tic twice
            print(now.strftime("%Y-%m-%d %H:%M:%S.%f")[:-4])
            print("msg.time", msg.timestamp)
            np_array = np.frombuffer(bytes(msg.data), dtype=np.uint8)
            frame = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
            self.last_img = frame
            self.time_last_img = tic
            self.warning_no_env_image_send = False
        else:
            if (
                tic - self.time_last_img > self.warning_no_env_image_dt
                and not self.warning_no_env_image_send
            ):
                print(
                    f"warning: client did not recieve env image in  last {self.warning_no_env_image_dt} [s]"
                )
                self.warning_no_env_image_send = True
                self.warning_no_env_image_send = False

        return self.last_img


model = FlexivModel()
robot = FlexivDDSClient()
camera = CameraDDSClient()
image = camera.get_env_image()
print(type(image))
plt.imshow(image)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/envs/flexivpy/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_80441/1995822004.py", line 17, in <module>
    import matplotlib.pyplot as plt
  File "/opt/conda/envs/flexivpy/lib/python3.10/site-packages/matplotlib/pyplot.py", line 55, in <module>
    import matplotlib.colorbar
  File "/opt/conda/envs/flexivpy/lib/python3.10/site-packages/matplotlib/colorbar.py", line 19, in <module>
    from matplotlib import _api, cbook, collections, cm, colors, contour, ticker
  File "/opt/conda/envs/flexivpy/lib/python3.10/site-packages/matplotlib/contour.py", line 15, in <module>
    from matplotlib.backend_bases import MouseButton
  File "/opt/conda/envs/flexivpy/lib/python3.10/site-packages/matplotlib/backend_bases.py", line 49, in <module>
    from matplotlib import (
  File "/opt/conda/envs/flexivpy/lib/python3.10/site-packages

In [ ]:
from FlexivPy.controllers.runners import NonBlockingRunner, blocking_runner
from FlexivPy.controllers.taskspace import DiffIKController


def get_T_from_controller(state):
    info = model.getInfo(np.array(state.q), np.array(state.dq))
    T0 = info["poses"]["link7"]
    # Initialize the desired pose
    x0, y0, z0 = 0.0, 0.0, 0.0
    R0 = np.eye(3)

    rate = 1.0 / 100.0

    joy_state = joy.getStates()
    left_joy = joy_state["left_joy"]
    right_joy = joy_state["right_joy"]

    if joy_state["right_bumper"] == 0:
        vx_cmd = right_joy[1]
        vy_cmd = right_joy[0]
        vz_cmd = left_joy[0]
        if np.abs(vx_cmd) < 0.1:
            vx_cmd = 0
        if np.abs(vy_cmd) < 0.1:
            vy_cmd = 0
        if np.abs(vz_cmd) < 0.1:
            vz_cmd = 0
        y0 = y0 + vy_cmd * rate
        x0 = x0 + vx_cmd * rate
        z0 = z0 - vz_cmd * rate
    else:
        wx_cmd = right_joy[1]
        wy_cmd = right_joy[0]
        wz_cmd = left_joy[0]
        if np.abs(wx_cmd) < 0.1:
            wx_cmd = 0
        if np.abs(wy_cmd) < 0.1:
            wy_cmd = 0
        if np.abs(wz_cmd) < 0.1:
            wz_cmd = 0
        cmd = np.array([wx_cmd, wy_cmd, wz_cmd])
        omega_hat = np.array(
            [[0, -cmd[2], cmd[1]], [cmd[2], 0, -cmd[0]], [-cmd[1], cmd[0], 0]]
        )
        R0 = R0 @ (np.eye(3) + omega_hat / 100)

    # time.sleep(0.01)
    T_cmd = T0 @ np.vstack(
        [np.hstack([R0, np.array([x0, y0, z0]).reshape(3, 1)]), np.array([0, 0, 0, 1])]
    )
    return T_cmd


task_controller = DiffIKController(
    model, T_cmd_fun=get_T_from_controller, dt=0.01, dq_max=1.0, control_mode="velocity"
)




max time reached


In [19]:




class CallbackFun(): 
    def __init__(self):
        self.cmds = []
        self.callback_dt = 0.01
        self.states = []
        self.imgs = []
        self.last_callback_t = -1

    def __call__(self, robot, cmd , _):
        tic = time.time()
        if tic - self.last_callback_t > self.callback_dt:
            last_callback_t = self.callback_dt
            self.cmds.append(cmd)
            self.states.append(robot.get_robot_state())
            self.imgs.append(camera.get_env_image())

callback = CallbackFun()


status = blocking_runner(robot, task_controller, timeout=20, callback=callback)


NameError: name 'camera' is not defined

In [ ]:
import cv2
import threading
import matplotlib.pyplot as plt
import time
class QVideoCapture:
    def __init__(self, name):
        self.cap = cv2.VideoCapture(name)
        self.lock = threading.Lock()
        self.running = True  # Flag to control the thread
        self.t = threading.Thread(target=self._reader)
        self.t.start()

    # Grab frames as soon as they are available
    def _reader(self):
        while self.running:
            with self.lock:
                ret = self.cap.grab()
                time.sleep(0.01)
                # sleep 
            if not ret:
                # If grabbing fails, perhaps the camera was disconnected
                break

    # Retrieve the latest frame
    def read(self):
        with self.lock:
            ret, frame = self.cap.retrieve()
        return ret, frame

    def release(self):
        self.running = False
        self.t.join()
        self.cap.release()


cap = QVideoCapture(5)
ret,frame = cap.read()

fig, ax = plt.subplots(1,3)
cap.release()

# convert the i
ax[0].imshow(frame)



In [27]:
# close the griper.
from FlexivPy.controllers.gripper import CloseGripper, OpenGripper
import time
time.sleep(10)
controller = CloseGripper(control_mode="velocity")
#controller = OpenGripper(control_mode="velocity")


status =  blocking_runner(
    robot, 
    controller, 
    timeout=10,
    callback=None
    )



goal reached


In [ ]:
import atexit
import subprocess

# Example: Starting a process


def cleanup():
    with open("tmp.log", "r") as f:
        f.write("hello\n")


# Register the cleanup function
atexit.register(cleanup)

In [ ]:
from FlexivPy.vision import RealSenseCamera
import signal

running = True



# get an image. 

# Camera to the right of the robot
# camera2 = RealSenseCamera(VGA=False, camera_serial_no='231622302407')

# Camera to the left of the robot
camera1 = RealSenseCamera(VGA=False, camera_serial_no='234222302193')





# get an image. 


In [ ]:
camera1.close()

In [ ]:
import matplotlib.pyplot as plt

import cv2
import numpy as np


image = camera1.color_frame
# Assuming 'image' is your NumPy array image
# Note: OpenCV uses width before height
new_width = image.shape[1] // 10
new_height = image.shape[0] // 10
new_dimensions = (new_width, new_height)

# Resize the image with INTER_AREA interpolation (best for shrinking)
image_resized = cv2.resize(image, new_dimensions, interpolation=cv2.INTER_AREA)


# Extract the central 64x64 pixels
height, width = image_resized.shape[:2]
central_size = 64
half_size = central_size // 2

center_y = height // 2
center_x = width // 2

start_y = max(0, center_y - half_size)
start_x = max(0, center_x - half_size)
end_y = start_y + central_size
end_x = start_x + central_size

end_y = min(height, end_y)
end_x = min(width, end_x)
start_y = max(0, end_y - central_size)
start_x = max(0, end_x - central_size)

central_area = image_resized[start_y:end_y, start_x:end_x]




fig, ax = plt.subplots(1,3)

print(camera1.color_frame.shape)
# convert the i
ax[0].imshow(camera1.color_frame)
ax[1].imshow(image_resized)
ax[2].imshow(central_area)

In [ ]:
qs = []
callback_dt = .1
last_callback_t = -1

cmds = []
states = []
imgs = []





runner = NonBlockingRunner(robot, task_controller, timeout=120. , callback=callback)

Exception in thread Thread-12 (loop):
Traceback (most recent call last):
  File "/opt/conda/envs/flexivpy/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/flexivpy/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/opt/conda/envs/flexivpy/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/FlexivPy/FlexivPy/controllers/runners.py", line 112, in loop
    cmd = self.controller.get_control(s, elapsed_time)
  File "/home/FlexivPy/FlexivPy/controllers/taskspace.py", line 83, in get_control
    dq_des = self.__call__(np.array(state.q), np.array(state.dq), self.T_cmd)
  File "/home/FlexivPy/FlexivPy/controllers/taskspace.py", line 63, in __call__
    raise Exception('The goal pose is too far!')
Exception: The goal pose is too far!


In [ ]:
import time
import numpy as np
import datetime





starting the controller


KeyboardInterrupt: 

In [ ]:
# lets store the data.
import pickle

fout = "tmp_data_v0.pkl"

with open(fout, "wb") as f:
    pickle.dump({"cmds": cmds, "states": states, "imgs": imgs}, f)